In [1]:
import sys
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from datetime import datetime
from itertools import combinations

PREDICCION_REAL = False
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {"booster":"gbtree", "max_depth":5, "eta": 0.4, "objective": "binary:logistic", "nthread":2,"gamma" : 0,"subsample" : 0.5}
RONDAS = 5

In [ ]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("entrenamiento-listo.csv")
test = pd.read_csv("test-listo.csv")

In [10]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
entrenamiento = entrenamiento.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])
test = test.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])

In [11]:
if(PREDICCION_REAL):
    objetivo = (entrenamiento['Stage'] == 'Closed Won').astype(int)
    entrenamiento = entrenamiento.drop(columns=['Stage','Fecha'])
    
    d_entrenamiento = xgb.DMatrix(entrenamiento.values, objetivo.values)
    d_prueba = xgb.DMatrix(test.values)
    
    bst = xgb.train(PARAMETROS, d_entrenamiento, RONDAS)
    preds = bst.predict(d_prueba)

    resultados = test[['Opportunity_ID']].copy()
    resultados['Target'] = pd.Series(preds)
    resultados = resultados.groupby('Opportunity_ID').mean()
    resultados = resultados.reset_index()
    resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))
    
    resultados.to_csv("prediccion.csv", index=False)
    resultados['Target'].value_counts()

In [12]:
if(PREDICCION_REAL): sys.exit()

In [13]:
entrenamiento_label = (entrenamiento['Stage'] == 'Closed Won').astype(int)
test_label          = (test['Stage'] == 'Closed Won').astype(int)

set_entrenamiento = xgb.DMatrix(entrenamiento.drop(columns=['Stage','Fecha']),label = entrenamiento_label)
set_test  = xgb.DMatrix(test .drop(columns=['Stage','Fecha']),label = test_label)
evaluacion = [(set_test, 'eval'), (set_entrenamiento, 'train')]

modelo = xgb.train(PARAMETROS, set_entrenamiento, RONDAS, evaluacion, early_stopping_rounds=2)

[17:39:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-logloss:0.68456	train-logloss:0.51107
[1]	eval-logloss:0.69631	train-logloss:0.42483
[2]	eval-logloss:0.71187	train-logloss:0.37418
[3]	eval-logloss:0.77043	train-logloss:0.34337
[4]	eval-logloss:0.81099	train-logloss:0.32031


In [14]:
prediccion = modelo.predict(set_test)

prediccion = [1 if i > .5 else 0 for i in prediccion]

def metricas(objetivo, prediccion):
    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return(metricas)

metrics = metricas(test_label, prediccion)
print("Prediccion Test")
[print(i) for i in metrics]
print(skl.metrics.log_loss(test_label,prediccion))

prediccion = modelo.predict(set_entrenamiento)
prediccion = [1 if i > .5 else 0 for i in prediccion]
metricas = metricas(entrenamiento_label, prediccion)
print("Prediccion Train")
[print(i) for i in metricas]

Prediccion Test
[[1645  134]
 [1925  846]]
0.5474725274725275
              precision    recall  f1-score   support

           0       0.46      0.92      0.62      1779
           1       0.86      0.31      0.45      2771

    accuracy                           0.55      4550
   macro avg       0.66      0.61      0.53      4550
weighted avg       0.71      0.55      0.52      4550

15.62976873476412
Prediccion Train
[[4682  864]
 [ 641 6115]]
0.8776621687530483
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      5546
           1       0.88      0.91      0.89      6756

    accuracy                           0.88     12302
   macro avg       0.88      0.87      0.88     12302
weighted avg       0.88      0.88      0.88     12302



[None, None, None]

In [15]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

SystemExit: 

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
min_log_loss = 1000
d = 1000
e = 1000
r = 1000
for depth in range(2,14):
    for eta in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for rondas in [10,20,30,40]:
            parametros = {"booster":"gbtree", "max_depth":depth, "eta": eta, "objective": "binary:logistic", "nthread":2}
            modelo = xgb.train(parametros, set_entrenamiento, rondas, evaluacion)
            prediccion = modelo.predict(set_test)
            prediccion = [1 if i > .5 else 0 for i in prediccion]
            log_loss = skl.metrics.log_loss(test_label,prediccion)
            if (log_loss < min_log_loss):
                min_log_loss = log_loss
                d = depth
                e = eta
                r = rondas


print("log: ",min_log_loss)                
print("depth: ",d)      
print("eta: ",e)      
print("rondas: ",r)

[17:40:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-logloss:0.67796	train-logloss:0.65963
[1]	eval-logloss:0.67488	train-logloss:0.63154
[2]	eval-logloss:0.66091	train-logloss:0.60694
[3]	eval-logloss:0.66185	train-logloss:0.58546
[4]	eval-logloss:0.66204	train-logloss:0.56742
[5]	eval-logloss:0.66426	train-logloss:0.55086
[6]	eval-logloss:0.65450	train-logloss:0.53140
[7]	eval-logloss:0.64180	train-logloss:0.51475
[8]	eval-logloss:0.63716	train-logloss:0.50322
[9]	eval-logloss:0.63606	train-logloss:0.49092
[17:40:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'e

[1]	eval-logloss:0.66166	train-logloss:0.58408
[2]	eval-logloss:0.66576	train-logloss:0.54790
[3]	eval-logloss:0.63853	train-logloss:0.51179
[4]	eval-logloss:0.63761	train-logloss:0.48882
[5]	eval-logloss:0.62893	train-logloss:0.46630
[6]	eval-logloss:0.64224	train-logloss:0.44973
[7]	eval-logloss:0.64836	train-logloss:0.43683
[8]	eval-logloss:0.64402	train-logloss:0.42499
[9]	eval-logloss:0.65562	train-logloss:0.41652
[10]	eval-logloss:0.67140	train-logloss:0.40909
[11]	eval-logloss:0.65867	train-logloss:0.40133
[12]	eval-logloss:0.67795	train-logloss:0.39498
[13]	eval-logloss:0.66964	train-logloss:0.38854
[14]	eval-logloss:0.66075	train-logloss:0.38386
[15]	eval-logloss:0.67874	train-logloss:0.37935
[16]	eval-logloss:0.68175	train-logloss:0.37575
[17]	eval-logloss:0.69507	train-logloss:0.37249
[18]	eval-logloss:0.70794	train-logloss:0.36930
[19]	eval-logloss:0.70979	train-logloss:0.36690
[20]	eval-logloss:0.70293	train-logloss:0.36408
[21]	eval-logloss:0.72242	train-logloss:0.36105
[

[11]	eval-logloss:0.66821	train-logloss:0.36875
[12]	eval-logloss:0.69442	train-logloss:0.36393
[13]	eval-logloss:0.72332	train-logloss:0.35964
[14]	eval-logloss:0.72698	train-logloss:0.35628
[15]	eval-logloss:0.74233	train-logloss:0.35308
[16]	eval-logloss:0.73876	train-logloss:0.35055
[17]	eval-logloss:0.75257	train-logloss:0.34790
[18]	eval-logloss:0.75168	train-logloss:0.34580
[19]	eval-logloss:0.77312	train-logloss:0.34336
[20]	eval-logloss:0.75967	train-logloss:0.33804
[21]	eval-logloss:0.76318	train-logloss:0.33634
[22]	eval-logloss:0.77511	train-logloss:0.33320
[23]	eval-logloss:0.77127	train-logloss:0.33133
[24]	eval-logloss:0.76898	train-logloss:0.32861
[25]	eval-logloss:0.77311	train-logloss:0.32729
[26]	eval-logloss:0.78964	train-logloss:0.32584
[27]	eval-logloss:0.78791	train-logloss:0.32483
[28]	eval-logloss:0.80067	train-logloss:0.32360
[29]	eval-logloss:0.80392	train-logloss:0.32240
[30]	eval-logloss:0.78412	train-logloss:0.31972
[31]	eval-logloss:0.78183	train-logloss:

[1]	eval-logloss:0.66402	train-logloss:0.49004
[2]	eval-logloss:0.62709	train-logloss:0.43828
[3]	eval-logloss:0.66510	train-logloss:0.41075
[4]	eval-logloss:0.66871	train-logloss:0.39494
[5]	eval-logloss:0.70556	train-logloss:0.38345
[6]	eval-logloss:0.71384	train-logloss:0.37526
[7]	eval-logloss:0.76781	train-logloss:0.36796
[8]	eval-logloss:0.81118	train-logloss:0.36153
[9]	eval-logloss:0.80445	train-logloss:0.34849
[10]	eval-logloss:0.80578	train-logloss:0.34096
[11]	eval-logloss:0.80467	train-logloss:0.33761
[12]	eval-logloss:0.83354	train-logloss:0.33335
[13]	eval-logloss:0.82875	train-logloss:0.33095
[14]	eval-logloss:0.83522	train-logloss:0.32795
[15]	eval-logloss:0.86569	train-logloss:0.32396
[16]	eval-logloss:0.84398	train-logloss:0.32056
[17]	eval-logloss:0.82590	train-logloss:0.31749
[18]	eval-logloss:0.86771	train-logloss:0.31487
[19]	eval-logloss:0.86210	train-logloss:0.31183
[17:40:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1

[21]	eval-logloss:0.87029	train-logloss:0.30505
[22]	eval-logloss:0.87811	train-logloss:0.30228
[23]	eval-logloss:0.88758	train-logloss:0.29987
[24]	eval-logloss:0.89386	train-logloss:0.29834
[25]	eval-logloss:0.89680	train-logloss:0.29733
[26]	eval-logloss:0.94215	train-logloss:0.29535
[27]	eval-logloss:0.93270	train-logloss:0.29295
[28]	eval-logloss:0.90991	train-logloss:0.29162
[29]	eval-logloss:0.90340	train-logloss:0.29055
[17:40:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-logloss:0.66124	train-logloss:0.54326
[1]	eval-logloss:0.67761	train-logloss:0.47075
[2]	eval-logloss:0.64021	train-logloss:0.42184
[3]	eval-logloss:0.68533	train-logloss:0.39912
[4]	eval-logloss:0.72968	train-logloss:0.38721
[5]	eval-logloss:0.7459

...

[1]	eval-logloss:0.67747	train-logloss:0.55782
[2]	eval-logloss:0.67483	train-logloss:0.50631
[3]	eval-logloss:0.67174	train-logloss:0.46087
[4]	eval-logloss:0.66800	train-logloss:0.42148
[5]	eval-logloss:0.67012	train-logloss:0.38711
[6]	eval-logloss:0.66959	train-logloss:0.35723
[7]	eval-logloss:0.66591	train-logloss:0.32994
[8]	eval-logloss:0.66880	train-logloss:0.30649
[9]	eval-logloss:0.67457	train-logloss:0.28524
[10]	eval-logloss:0.67626	train-logloss:0.26571
[11]	eval-logloss:0.67964	train-logloss:0.24851
[12]	eval-logloss:0.68300	train-logloss:0.23224
[13]	eval-logloss:0.68531	train-logloss:0.21789
[14]	eval-logloss:0.69380	train-logloss:0.20469
[15]	eval-logloss:0.69643	train-logloss:0.19214
[16]	eval-logloss:0.69953	train-logloss:0.18058
[17]	eval-logloss:0.70459	train-logloss:0.17093
[18]	eval-logloss:0.70788	train-logloss:0.16133
[19]	eval-logloss:0.70632	train-logloss:0.15240
[20]	eval-logloss:0.70549	train-logloss:0.14451
[21]	eval-logloss:0.70985	train-logloss:0.13690
[

[11]	eval-logloss:0.74645	train-logloss:0.12069
[12]	eval-logloss:0.75914	train-logloss:0.10951
[13]	eval-logloss:0.75714	train-logloss:0.09969
[14]	eval-logloss:0.77219	train-logloss:0.09202
[15]	eval-logloss:0.77734	train-logloss:0.08538
[16]	eval-logloss:0.78234	train-logloss:0.08037
[17]	eval-logloss:0.78814	train-logloss:0.07560
[18]	eval-logloss:0.79406	train-logloss:0.07056
[19]	eval-logloss:0.79380	train-logloss:0.06637
[20]	eval-logloss:0.80697	train-logloss:0.06291
[21]	eval-logloss:0.80633	train-logloss:0.05919
[22]	eval-logloss:0.82018	train-logloss:0.05676
[23]	eval-logloss:0.83678	train-logloss:0.05316
[24]	eval-logloss:0.84617	train-logloss:0.05122
[25]	eval-logloss:0.85600	train-logloss:0.04830
[26]	eval-logloss:0.85935	train-logloss:0.04619
[27]	eval-logloss:0.86315	train-logloss:0.04395
[28]	eval-logloss:0.86202	train-logloss:0.04288
[29]	eval-logloss:0.87137	train-logloss:0.04134
[30]	eval-logloss:0.87361	train-logloss:0.04043
[31]	eval-logloss:0.88130	train-logloss:

[14]	eval-logloss:0.85635	train-logloss:0.03035
[15]	eval-logloss:0.84999	train-logloss:0.02790
[16]	eval-logloss:0.86383	train-logloss:0.02592
[17]	eval-logloss:0.89399	train-logloss:0.02419
[18]	eval-logloss:0.89349	train-logloss:0.02241
[19]	eval-logloss:0.90119	train-logloss:0.02098
[20]	eval-logloss:0.90255	train-logloss:0.01982
[21]	eval-logloss:0.90973	train-logloss:0.01818
[22]	eval-logloss:0.91317	train-logloss:0.01687
[23]	eval-logloss:0.91364	train-logloss:0.01597
[24]	eval-logloss:0.91310	train-logloss:0.01498
[25]	eval-logloss:0.91387	train-logloss:0.01410
[26]	eval-logloss:0.91114	train-logloss:0.01322
[27]	eval-logloss:0.92999	train-logloss:0.01266
[28]	eval-logloss:0.95430	train-logloss:0.01202
[29]	eval-logloss:0.94418	train-logloss:0.01159
[17:52:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'lo

[24]	eval-logloss:1.07733	train-logloss:0.01200
[25]	eval-logloss:1.08464	train-logloss:0.01125
[26]	eval-logloss:1.09948	train-logloss:0.01064
[27]	eval-logloss:1.10525	train-logloss:0.01004
[28]	eval-logloss:1.09837	train-logloss:0.00959
[29]	eval-logloss:1.10439	train-logloss:0.00918
[30]	eval-logloss:1.09670	train-logloss:0.00879
[31]	eval-logloss:1.10665	train-logloss:0.00841
[32]	eval-logloss:1.12234	train-logloss:0.00807
[33]	eval-logloss:1.13395	train-logloss:0.00777
[34]	eval-logloss:1.14625	train-logloss:0.00746
[35]	eval-logloss:1.15614	train-logloss:0.00719
[36]	eval-logloss:1.14389	train-logloss:0.00692
[37]	eval-logloss:1.15601	train-logloss:0.00671
[38]	eval-logloss:1.16676	train-logloss:0.00650
[39]	eval-logloss:1.18097	train-logloss:0.00630
[17:53:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'lo

[8]	eval-logloss:1.04893	train-logloss:0.03231
[9]	eval-logloss:1.10344	train-logloss:0.02713
[10]	eval-logloss:1.09456	train-logloss:0.02326
[11]	eval-logloss:1.10255	train-logloss:0.02047
[12]	eval-logloss:1.13509	train-logloss:0.01826
[13]	eval-logloss:1.14220	train-logloss:0.01599
[14]	eval-logloss:1.14145	train-logloss:0.01468
[15]	eval-logloss:1.15915	train-logloss:0.01333
[16]	eval-logloss:1.18440	train-logloss:0.01246
[17]	eval-logloss:1.19369	train-logloss:0.01155
[18]	eval-logloss:1.19834	train-logloss:0.01067
[19]	eval-logloss:1.19348	train-logloss:0.00994
[17:53:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-logloss:0.73675	train-logloss:0.28957
[1]	eval-logloss:0.81850	train-logloss:0.17568
[2]	eval-logloss:0.831